For the sake of practice and to equip yourself with additional information, I suggest you to use another API here. What about using `AlphaVantage`. Alpha Vantage is a data providing company partnered with major exchanges and institutions. Using Alpha Vantage' API, it is possible to access stock prices, stock fundamentals, Forex information and many more with different time interval- e.g. intraday, daily, weekly, an so on. For more information, please see https://www.alphavantage.co/support/#support[Alpha Vantage's website.]

Let me change the tickers so that we can analyze different companies. This time, let's extract the stock prices of two social media companies, which are:
* Twitter
* Facebook

Having listed some relevant variables, we are now able to run the `APT` analysis. We could proceed by retrieving the data first. Again, we refer to the relate APIs to extract these data. Thanks to `Yahoo Finance` and `FRED`, we have stock prices, risk free rate, and stock market index (S&P-500) from the preceding part.

Let's import all the necessary libraries first.

In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from fredapi import Fred

In [2]:
def getMonthlyData(symbol):
        parameters = {'function': 'TIME_SERIES_MONTHLY_ADJUSTED', 'symbol': symbol,
                       'outputsize':'full',
                       'datatype': 'csv', 
                       'apikey': 'LL1WA15IW41XV2T2'}

        response = requests.get('https://www.alphavantage.co/query', params=parameters)

        # Process the CSV file retrieved
        csvText = StringIO(response.text)
        data = pd.read_csv(csvText, index_col='timestamp')
        return data

Before starting, here is a little trick for you. The ticker of the `S&P-500` is `SPY` in Alpha Vantage API.

In [3]:
tickers = ["TWTR", "FB", "SPY"]
df = []
for i in tickers:
    df.append(getMonthlyData(i)[::-1]['close']['2015-01-01':'2021-05-01'])
stocks = pd.DataFrame(df).T
stocks.columns = tickers

In [4]:
stocks.head()


,TWTR,FB,SPY
timestamp,,,
2015-01-30,37.53,75.910,199.45
2015-02-27,48.08,78.970,210.66
2015-03-31,50.08,82.215,206.43
2015-04-30,38.96,78.770,208.46
2015-05-29,36.67,79.190,211.14


As per the article of Chen, Roll and Ross (1986), I want to pick the following three variables:
    
* Growth rate in industrial production

* Inflation rate

* Stock Index

These variables will be your independent variables as they have appeared to explain a significant portion of the time-series variability of stock returns.

At this point, to extract the macroeconomic variable, `FRED API` would be a convenient database:

In [5]:
fred = Fred(api_key='78b14ec6ba46f484b94db43694468bb1')

In [6]:
fred.search('industrial')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
INDPRO,INDPRO,2021-05-20,2021-05-20,Industrial Production: Total Index,1919-01-01,2021-04-01,Monthly,M,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2021-05-14 08:19:02-05:00,85,The Industrial Production Index (INDPRO) is an...
IPB50001N,IPB50001N,2021-05-20,2021-05-20,Industrial Production: Total Index,1919-01-01,2021-04-01,Monthly,M,Index 2012=100,Index 2012=100,Not Seasonally Adjusted,NSA,2021-05-14 08:19:21-05:00,43,The industrial production (IP) index measures ...
IPB50001SQ,IPB50001SQ,2021-05-20,2021-05-20,Industrial Production: Total Index,1919-01-01,2021-01-01,Quarterly,Q,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2021-05-14 08:18:07-05:00,28,The industrial production (IP) index measures ...
IPB50001NQ,IPB50001NQ,2021-05-20,2021-05-20,Industrial Production: Total Index,1919-01-01,2021-01-01,Quarterly,Q,Index 2012=100,Index 2012=100,Not Seasonally Adjusted,NSA,2021-05-14 08:18:24-05:00,15,The industrial production (IP) index measures ...
IPMAN,IPMAN,2021-05-20,2021-05-20,Industrial Production: Manufacturing (NAICS),1972-01-01,2021-04-01,Monthly,M,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2021-05-14 08:18:02-05:00,63,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IPB53200SQ,IPB53200SQ,2021-05-20,2021-05-20,Industrial Production: Non-Durable Goods Mater...,1954-01-01,2021-01-01,Quarterly,Q,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2021-05-14 08:22:53-05:00,1,The industrial production (IP) index measures ...
IPB53200NQ,IPB53200NQ,2021-05-20,2021-05-20,Industrial Production: Non-Durable Goods Mater...,1954-01-01,2021-01-01,Quarterly,Q,Index 2012=100,Index 2012=100,Not Seasonally Adjusted,NSA,2021-05-14 08:22:53-05:00,1,The industrial production (IP) index measures ...
IPG32513S,IPG32513S,2021-05-20,2021-05-20,Industrial Production: Manufacturing: Non-Dura...,1972-01-01,2021-04-01,Monthly,M,Index 2012=100,Index 2012=100,Seasonally Adjusted,SA,2021-05-14 08:20:29-05:00,1,The industrial production (IP) index measures ...


In [7]:
ind = fred.get_series('INDPRO')
ind = ind['2015-01-01':'2021-05-01']

In [8]:
ind.head()

2015-01-01    105.9806
2015-02-01    105.4425
2015-03-01    105.1464
2015-04-01    104.5272
2015-05-01    104.0742
dtype: float64

In [9]:
fred.search('potential inflation')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
FEDTARMDLR,FEDTARMDLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2012-01-25,2021-03-17,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2021-03-17 15:57:05-05:00,44,The longer-run projections are the rates of gr...
GDPC1CTMLR,GDPC1CTMLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:11-05:00,27,The longer-run projections are the rates of gr...
GDPC1MDLR,GDPC1MDLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:20-05:00,27,The longer-run projections are the rates of gr...
UNRATECTMLR,UNRATECTMLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,"Fourth Quarter, Percent","Fourth Qtr., %",Not Seasonally Adjusted,NSA,2021-03-17 15:57:14-05:00,18,The longer-run projections are the rates of gr...
PCECTPICTMLR,PCECTPICTMLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:19-05:00,12,The longer-run projections are the rates of gr...
GDPC1RHLR,GDPC1RHLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:35-05:00,11,The longer-run projections are the rates of gr...
PCECTPIRLLR,PCECTPIRLLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:22-05:00,10,The longer-run projections are the rates of gr...
UNRATEMDLR,UNRATEMDLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2015-06-17,2021-03-17,Not Applicable,NA,Percent,%,Not Seasonally Adjusted,NSA,2021-03-17 15:57:37-05:00,9,The longer-run projections are the rates of gr...
PCECTPICTLLR,PCECTPICTLLR,2021-05-20,2021-05-20,Longer Run FOMC Summary of Economic Projection...,2009-02-18,2021-03-17,Not Applicable,NA,Fourth Quarter to Fourth Quarter Percent Change,Fourth Qtr. to Fourth Qtr. % Chg.,Not Seasonally Adjusted,NSA,2021-03-17 15:57:25-05:00,8,The longer-run projections are the rates of gr...


In [10]:
inf = fred.get_series('CPIEALL')
inf = inf['2015-01-01':'2021-05-01']

As you are working with monthly data, it does not make sense to use risk free rate (`DGS3MO`) that you used in CAPM as it is daily rate. So, let's pick a monthly version of it, which is `GS3M`.Finally, you need to have risk free rate in APT analysis, as well. To do that, you can make use of `FRED` again. 

In [11]:
fred.search('risk free')

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
DGS10,DGS10,2021-05-20,2021-05-20,10-Year Treasury Constant Maturity Rate,1962-01-02,2021-05-18,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-05-19 15:23:09-05:00,97,For further information regarding treasury con...
GS10,GS10,2021-05-20,2021-05-20,10-Year Treasury Constant Maturity Rate,1953-04-01,2021-04-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-05-03 15:23:21-05:00,78,Averages of business days. For further informa...
WGS10YR,WGS10YR,2021-05-20,2021-05-20,10-Year Treasury Constant Maturity Rate,1962-01-05,2021-05-14,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2021-05-17 15:23:17-05:00,62,Averages of business days. For further informa...
DGS1,DGS1,2021-05-20,2021-05-20,1-Year Treasury Constant Maturity Rate,1962-01-02,2021-05-18,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-05-19 15:23:07-05:00,80,For further information regarding treasury con...
WGS1YR,WGS1YR,2021-05-20,2021-05-20,1-Year Treasury Constant Maturity Rate,1962-01-05,2021-05-14,"Weekly, Ending Friday",W,Percent,%,Not Seasonally Adjusted,NSA,2021-05-17 15:23:20-05:00,59,Averages of business days. For further informa...
GS1,GS1,2021-05-20,2021-05-20,1-Year Treasury Constant Maturity Rate,1953-04-01,2021-04-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-05-03 15:23:37-05:00,55,Averages of business days. For further informa...
DGS5,DGS5,2021-05-20,2021-05-20,5-Year Treasury Constant Maturity Rate,1962-01-02,2021-05-18,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-05-19 15:23:23-05:00,79,For further information regarding treasury con...
DGS30,DGS30,2021-05-20,2021-05-20,30-Year Treasury Constant Maturity Rate,1977-02-15,2021-05-18,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2021-05-19 15:23:20-05:00,78,Yields on actively traded non-inflation-indexe...
GS5,GS5,2021-05-20,2021-05-20,5-Year Treasury Constant Maturity Rate,1953-04-01,2021-04-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2021-05-03 15:23:34-05:00,53,Averages of business days. For further informa...


In [12]:
risk_free = fred.get_series('GS1')
risk_free = risk_free['2015-01-01':'2021-05-01']